In [1]:
using Pkg
Pkg.activate("D:\\Science\\Code\\Julia\\envr")

  Activating project at `D:\Science\Code\Julia\envr`


In [1]:
using StaticArrays, DifferentialEquations, DynamicalSystems, CairoMakie, JLD

In [2]:
function TM(u, p, t)

    U(y) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
    
    # u[1] - E; u[2] - x; u[3] - y
    # α - 1, τ - 2, τD - 3, τy - 4, J - 5, xthr - 6, ythr - 7, U0 - 8, ΔU0 - 9, β - 10, I0 -11

    du1 = (-u[1] + p[1] * log( 1.0 + exp( (p[5] * U(u[3]) * u[2] * u[1] + p[11]  ) / (p[1]) ) ) ) / p[2]
    du2 = (1.0 - u[2])/p[3] - U(u[3])*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2])
    return SA[du1, du2, du3]
end;

In [3]:
function output(idx_I0, idx_U0, I0, U0, u0)
    println("Индекс I0: $idx_I0; I0: $I0");flush(stdout)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output(idx_U0,U0, u0)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output_end_iter(ΛΛ, u0_lc)
    println("Спектр: $(ΛΛ)");flush(stdout)
    println("Последняя точка: $(u0_lc)");flush(stdout)
end
function separate()
    println();flush(stdout)
    println("---------");flush(stdout)
end;

In [17]:
t = 1000.0; tt = 250.0; tstep = 0.001
time_attract = 100.0
trange = range(0.0, t, step = tstep)
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [18]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58;  U0 = 0.3;  const ΔU0 = 0.305; I0 = -1.40;

In [19]:
u0 = SA[17.6055, 0.6181, 0.429232]

3-element SVector{3, Float64} with indices SOneTo(3):
 17.6055
  0.6181
  0.429232

In [20]:
I0range = range( -1.40, -1.709, length = 201 )
U0range = range( 0.3,  0.47, length = 201)

0.3:0.00085:0.47

In [21]:
Λs = zeros(length(I0range), length(U0range), 3)
u0s = zeros(length(I0range), length(U0range), 3);

In [22]:
function init_ds(U0_, u0_lc)
    p = SA[α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0range[1]]
    ds = ContinuousDynamicalSystem(TM, u0_lc, p)
    return ds
end
function init_ds(U0_, I0_, u0_lc)
    p = SA[α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_]
    ds = ContinuousDynamicalSystem(TM, u0_lc, p)
    return ds
end
function goto_attractor(ds_, time_attract)
    tr = trajectory(ds_, time_attract; Ttr = tt, diffeq = integ_set)
    u0_lc = tr[end]
    return u0_lc
end
function spectrum(ds_)
    ΛΛ = lyapunovspectrum(ds_, t; diffeq = integ_set)
    return ΛΛ
end
function save_output(idx_U0, ΛΛ, u0_lc)
    Λs[1, idx_U0, :] = ΛΛ
    u0s[1, idx_U0, :] = u0_lc
end
function save_output(index_I0, idx_U0, ΛΛ, u0_lc)
    Λs[index_I0, idx_U0, :] = ΛΛ
    u0s[index_I0, idx_U0, :] = u0_lc
end
function save_tofile()
    save("LSs_200x200.jld", "data", Λs)
    save("u0s_200x200.jld", "data", u0s)
end

save_tofile (generic function with 1 method)

In [23]:
for (idx_U0, U0_) in enumerate(U0range)
    
    if idx_U0 == 1 # Проверка на первую итерацию
        global u0_lc = u0
    end
    
    output(idx_U0, U0_, u0_lc) # Вывод
    ds = init_ds(U0_, u0_lc) # Инициализация системы
    u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
    ΛΛ = spectrum(ds) # Вычисление спектра
    save_output(idx_U0, ΛΛ, u0_lc) # Сохранение значений
    output_end_iter(ΛΛ, u0_lc) # Вывод
    save_tofile()
    separate()
    
end

Индекс U0: 1; U0: 0.3
Начальные условия: [17.6055, 0.6181, 0.429232]
Спектр: [0.004162543159739812, -2.768621222382334, -2.7798756740727404]
Последняя точка: [16.898370136765177, 0.6822784289462809, 0.4324577396837501]

---------
Индекс U0: 2; U0: 0.30085
Начальные условия: [16.898370136765177, 0.6822784289462809, 0.4324577396837501]
Спектр: [0.003547736104334139, -2.7764877758703492, -2.780654825581549]
Последняя точка: [4.5139640726808485, 0.7785677736592077, 0.4175214356212717]

---------
Индекс U0: 3; U0: 0.3017
Начальные условия: [4.5139640726808485, 0.7785677736592077, 0.4175214356212717]
Спектр: [0.0049348569680253765, -2.7821537407882717, -2.786649942708512]
Последняя точка: [4.442444697641834, 0.8239052696578935, 0.4242492588718571]

---------
Индекс U0: 4; U0: 0.30255
Начальные условия: [4.442444697641834, 0.8239052696578935, 0.4242492588718571]
Спектр: [0.005338177422120633, -2.7869794461766255, -2.792979898876534]
Последняя точка: [4.837178342481933, 0.7624392059066314, 0.4


---------
Индекс U0: 32; U0: 0.32635
Начальные условия: [13.483023417254634, 0.7492169228167114, 0.4268202663775662]
Спектр: [0.003716768731554144, -1.8790825518645724, -3.979305579247062]
Последняя точка: [17.10888768136173, 0.5834140288608668, 0.4202125636442832]

---------
Индекс U0: 33; U0: 0.3272
Начальные условия: [17.10888768136173, 0.5834140288608668, 0.4202125636442832]
Спектр: [0.00463541303715809, -1.8545147886011715, -4.012218787982347]
Последняя точка: [8.322346507114693, 0.8054624256134673, 0.424579980218005]

---------
Индекс U0: 34; U0: 0.32805
Начальные условия: [8.322346507114693, 0.8054624256134673, 0.424579980218005]
Спектр: [0.0033831122400914053, -1.8382566144033123, -4.03760136768816]
Последняя точка: [15.870864370074932, 0.5819195051497963, 0.4186443464356489]

---------
Индекс U0: 35; U0: 0.3289
Начальные условия: [15.870864370074932, 0.5819195051497963, 0.4186443464356489]
Спектр: [0.005032596176936192, -1.8185275299079284, -4.067160422023611]
Последняя точка

Последняя точка: [13.06556329271514, 0.5859743478658809, 0.41002104732207006]

---------
Индекс U0: 63; U0: 0.3527
Начальные условия: [13.06556329271514, 0.5859743478658809, 0.41002104732207006]
Спектр: [0.0050721547320534376, -1.829637464841223, -4.268954741423069]
Последняя точка: [17.74924989652397, 0.6958245979371254, 0.4196840268832332]

---------
Индекс U0: 64; U0: 0.35355
Начальные условия: [17.74924989652397, 0.6958245979371254, 0.4196840268832332]
Спектр: [0.004202976628544379, -1.8475113157248173, -4.257988434176819]
Последняя точка: [13.562865020745564, 0.5819603513078792, 0.41000131830976466]

---------
Индекс U0: 65; U0: 0.3544
Начальные условия: [13.562865020745564, 0.5819603513078792, 0.41000131830976466]
Спектр: [0.004472003920789073, -1.8613658369730042, -4.248694202541635]
Последняя точка: [8.407644374845658, 0.8064069257380879, 0.4177379458453216]

---------
Индекс U0: 66; U0: 0.35525
Начальные условия: [8.407644374845658, 0.8064069257380879, 0.4177379458453216]
Спек

Спектр: [0.0036801260277125867, -3.109188013410243, -3.1202911494419]
Последняя точка: [3.751227522007566, 0.805905085061033, 0.3999702124745171]

---------
Индекс U0: 94; U0: 0.37905
Начальные условия: [3.751227522007566, 0.805905085061033, 0.3999702124745171]
Спектр: [0.006640561997638133, -3.111469194530246, -3.1208007742790946]
Последняя точка: [4.574095894656344, 0.8311997730795118, 0.406412787357213]

---------
Индекс U0: 95; U0: 0.3799
Начальные условия: [4.574095894656344, 0.8311997730795118, 0.406412787357213]
Спектр: [0.0060798797183696405, -3.1158233079279474, -3.119466171872614]
Последняя точка: [11.36633214452056, 0.5986014733827543, 0.4012995194998914]

---------
Индекс U0: 96; U0: 0.38075
Начальные условия: [11.36633214452056, 0.5986014733827543, 0.4012995194998914]
Спектр: [0.004067043804448434, -3.1150816809603263, -3.1217619887623105]
Последняя точка: [21.133924694037713, 0.5996737951462554, 0.40954093702724664]

---------
Индекс U0: 97; U0: 0.3816
Начальные условия: 

Начальные условия: [3.876886255033725, 0.8071433014022357, 0.3937118058298178]
Спектр: [0.002192854228744843, -3.119119682737881, -3.1246748441832812]
Последняя точка: [20.463032110874515, 0.637641852796912, 0.40417704810311805]

---------
Индекс U0: 125; U0: 0.4054
Начальные условия: [20.463032110874515, 0.637641852796912, 0.40417704810311805]
Спектр: [0.001287046130992168, -3.1202966004576878, -3.123024483364825]
Последняя точка: [4.013368758916149, 0.7911368668323054, 0.3913782429319895]

---------
Индекс U0: 126; U0: 0.40625
Начальные условия: [4.013368758916149, 0.7911368668323054, 0.3913782429319895]
Спектр: [0.001108889590783403, -3.1174438007834846, -3.1219456755454384]
Последняя точка: [7.1948800920994955, 0.6730031709403215, 0.3898155775769507]

---------
Индекс U0: 127; U0: 0.4071
Начальные условия: [7.1948800920994955, 0.6730031709403215, 0.3898155775769507]
Спектр: [0.006879995742727584, -3.1196895803405615, -3.1219468913966235]
Последняя точка: [8.36062943054507, 0.647260

Последняя точка: [7.915415944078202, 0.8006760961719738, 0.3948332789730233]

---------
Индекс U0: 155; U0: 0.4309
Начальные условия: [7.915415944078202, 0.8006760961719738, 0.3948332789730233]
Спектр: [0.003922931954382024, -3.0755978221372313, -3.0779013382293714]
Последняя точка: [7.596903603644828, 0.6718006806605765, 0.38238639215654424]

---------
Индекс U0: 156; U0: 0.43175
Начальные условия: [7.596903603644828, 0.6718006806605765, 0.38238639215654424]
Спектр: [0.006219896628226533, -3.0728578974352345, -3.0771241287833853]
Последняя точка: [7.611844578407852, 0.8025794681884458, 0.3939011640013932]

---------
Индекс U0: 157; U0: 0.4326
Начальные условия: [7.611844578407852, 0.8025794681884458, 0.3939011640013932]
Спектр: [0.0032279689891741466, -3.069540473003008, -3.075381505153418]
Последняя точка: [9.584419775489506, 0.6350604448352325, 0.38383691823555466]

---------
Индекс U0: 158; U0: 0.43345
Начальные условия: [9.584419775489506, 0.6350604448352325, 0.38383691823555466]


Спектр: [0.0025829116030639337, -2.149747816363986, -3.902020141794147]
Последняя точка: [8.427041598427534, 0.7795339313509513, 0.38193478282605525]

---------
Индекс U0: 186; U0: 0.45725
Начальные условия: [8.427041598427534, 0.7795339313509513, 0.38193478282605525]
Спектр: [0.002783935095529815, -1.8862564637408525, -4.167147233503345]
Последняя точка: [10.09130462566181, 0.6538602258596353, 0.3735950645906199]

---------
Индекс U0: 187; U0: 0.4581
Начальные условия: [10.09130462566181, 0.6538602258596353, 0.3735950645906199]
Спектр: [0.0027410883468134166, -1.6702672538119887, -4.383129965473053]
Последняя точка: [13.335189531841603, 0.6322798312962741, 0.37663413270443236]

---------
Индекс U0: 188; U0: 0.45895
Начальные условия: [13.335189531841603, 0.6322798312962741, 0.37663413270443236]
Спектр: [0.0006951980141468717, -1.4823165805072174, -4.570826426836524]
Последняя точка: [6.286070068607791, 0.7370774757431762, 0.3699790048400178]

---------
Индекс U0: 189; U0: 0.4598
Начал

In [ ]:
for (idx_U0, U0_) in enumerate(U0range)
    for (idx_I0, I0_) in enumerate(I0range)
        
        if idx_I0 == 1
            continue
        end
        
        u0_lc = u0s[idx_I0 - 1, idx_U0, :]
        output(idx_I0, idx_U0, I0_, U0_, u0_lc)
        ds = init_ds(U0_, I0_, u0_lc)
        u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
        ΛΛ = spectrum(ds) # Вычисление спектра
        save_output(idx_I0, idx_U0, ΛΛ, u0_lc)
        output_end_iter(ΛΛ, u0_lc)
        save_tofile()
        separate()
    end
end

Индекс I0: 2; I0: -1.401545
Индекс U0: 1; U0: 0.3
Начальные условия: [16.898370136765177, 0.6822784289462809, 0.4324577396837501]
Спектр: [0.0037077894947957243, -2.7608365639360772, -2.7701269589680275]
Последняя точка: [4.987993480858915, 0.7574549013073204, 0.41677544275335277]

---------
Индекс I0: 3; I0: -1.40309
Индекс U0: 1; U0: 0.3
Начальные условия: [4.987993480858915, 0.7574549013073204, 0.41677544275335277]
Спектр: [0.005853345864113327, -2.757141183856239, -2.759715253022565]
Последняя точка: [4.374674653076458, 0.7867560214802731, 0.4184972704633458]

---------
Индекс I0: 4; I0: -1.404635
Индекс U0: 1; U0: 0.3
Начальные условия: [4.374674653076458, 0.7867560214802731, 0.4184972704633458]
Спектр: [0.005946419648979892, -2.748600392192197, -2.7517294778566823]
Последняя точка: [12.206762936268225, 0.6148287231606467, 0.4232325336329453]

---------
Индекс I0: 5; I0: -1.40618
Индекс U0: 1; U0: 0.3
Начальные условия: [12.206762936268225, 0.6148287231606467, 0.4232325336329453]


Начальные условия: [17.27686187653327, 0.6029995890220272, 0.4300776003997056]
Спектр: [0.0019523418798265263, -1.328965239907767, -3.7487068258851597]
Последняя точка: [4.070635655755762, 0.8187724987056544, 0.4242249741755632]

---------
Индекс I0: 31; I0: -1.44635
Индекс U0: 1; U0: 0.3
Начальные условия: [4.070635655755762, 0.8187724987056544, 0.4242249741755632]
Спектр: [0.004950824874636038, -1.2768320125907158, -3.7881044014617107]
Последняя точка: [5.899506426390269, 0.823311010273663, 0.43114546879882454]

---------
Индекс I0: 32; I0: -1.447895
Индекс U0: 1; U0: 0.3
Начальные условия: [5.899506426390269, 0.823311010273663, 0.43114546879882454]
Спектр: [0.005310164398854085, -1.2210999697154008, -3.8283360609891015]
Последняя точка: [4.639921908647078, 0.769229884697525, 0.4187755133716019]

---------
Индекс I0: 33; I0: -1.44944
Индекс U0: 1; U0: 0.3
Начальные условия: [4.639921908647078, 0.769229884697525, 0.4187755133716019]
Спектр: [0.004682730388134236, -1.1686876504842452, 

Спектр: [0.0036902305982416876, -0.15825005534785805, -4.454558423717784]
Последняя точка: [17.46829032348444, 0.5964161709955557, 0.4319598455686052]

---------
Индекс I0: 59; I0: -1.48961
Индекс U0: 1; U0: 0.3
Начальные условия: [17.46829032348444, 0.5964161709955557, 0.4319598455686052]
Спектр: [0.0024867014119356445, -0.12597480917816192, -4.471835688537799]
Последняя точка: [13.362011136194276, 0.5980215136668187, 0.42780441060717656]

---------
Индекс I0: 60; I0: -1.491155
Индекс U0: 1; U0: 0.3
Начальные условия: [13.362011136194276, 0.5980215136668187, 0.42780441060717656]
Спектр: [0.002889228779162092, -0.09269329996961662, -4.487146311827532]
Последняя точка: [6.67550718161027, 0.8187880069324178, 0.4344798199386322]

---------
Индекс I0: 61; I0: -1.4927
Индекс U0: 1; U0: 0.3
Начальные условия: [6.67550718161027, 0.8187880069324178, 0.4344798199386322]
Спектр: [0.0036717088371568773, -0.06473095111890499, -4.500549280850972]
Последняя точка: [18.789566979370257, 0.623649136305

Последняя точка: [10.86946959314275, 0.6032098754441424, 0.4313398767825546]

---------
Индекс I0: 87; I0: -1.53287
Индекс U0: 1; U0: 0.3
Начальные условия: [10.86946959314275, 0.6032098754441424, 0.4313398767825546]
Спектр: [0.005426969661905006, -2.163774211103072, -2.171064583895399]
Последняя точка: [20.99233636836888, 0.5885476049491966, 0.44118562972252473]

---------
Индекс I0: 88; I0: -1.534415
Индекс U0: 1; U0: 0.3
Начальные условия: [20.99233636836888, 0.5885476049491966, 0.44118562972252473]
Спектр: [0.006189663473640857, -2.160009972029116, -2.165440682858639]
Последняя точка: [3.5840366382125626, 0.8067176340965074, 0.4214388698857414]

---------
Индекс I0: 89; I0: -1.53596
Индекс U0: 1; U0: 0.3
Начальные условия: [3.5840366382125626, 0.8067176340965074, 0.4214388698857414]
Спектр: [0.0047810256767541, -2.1538353473366496, -2.1604172788358937]
Последняя точка: [5.296909040054898, 0.757657162743853, 0.4252840344895738]

---------
Индекс I0: 90; I0: -1.537505
Индекс U0: 1; U


---------
Индекс I0: 115; I0: -1.57613
Индекс U0: 1; U0: 0.3
Начальные условия: [14.120255978724852, 0.5609824701243377, 0.44020790062959414]
Спектр: [0.0004829391986208359, -0.1593504636037762, -3.8366018142311975]
Последняя точка: [13.978067737521663, 0.6556622549882317, 0.42834046987106467]

---------
Индекс I0: 116; I0: -1.577675
Индекс U0: 1; U0: 0.3
Начальные условия: [13.978067737521663, 0.6556622549882317, 0.42834046987106467]
Спектр: [0.19956046986680678, 9.952038351072423e-5, -4.179314547680067]
Последняя точка: [5.703279859016976, 0.7387066611413308, 0.4179481841292046]

---------
Индекс I0: 117; I0: -1.57922
Индекс U0: 1; U0: 0.3
Начальные условия: [5.703279859016976, 0.7387066611413308, 0.4179481841292046]
Спектр: [0.3011423055467429, -0.0025544490851078603, -4.2684691586651935]
Последняя точка: [7.13540792512351, 0.7039344808826863, 0.4185870357143433]

---------
Индекс I0: 118; I0: -1.580765
Индекс U0: 1; U0: 0.3
Начальные условия: [7.13540792512351, 0.7039344808826863,

...


---------
Индекс I0: 25; I0: -1.43708
Индекс U0: 8; U0: 0.30595
Начальные условия: [14.23787303160134, 0.5953358778982589, 0.4247218151074689]
Спектр: [0.0045162589035928746, -1.2327076117931806, -4.0078678538150845]
Последняя точка: [5.305535593818996, 0.7403353862347282, 0.4158928506652169]

---------
Индекс I0: 26; I0: -1.438625
Индекс U0: 8; U0: 0.30595
Начальные условия: [5.305535593818996, 0.7403353862347282, 0.4158928506652169]
Спектр: [0.005718690050531577, -1.182560776455028, -4.042393355409368]
Последняя точка: [7.943313293825443, 0.6681691779268418, 0.41797513609824344]

---------
Индекс I0: 27; I0: -1.44017
Индекс U0: 8; U0: 0.30595
Начальные условия: [7.943313293825443, 0.6681691779268418, 0.41797513609824344]
Спектр: [0.005342230004892705, -1.1303085533146235, -4.076727951570263]
Последняя точка: [5.223537147191629, 0.8281508429942855, 0.42808898776538873]

---------
Индекс I0: 28; I0: -1.441715
Индекс U0: 8; U0: 0.30595
Начальные условия: [5.223537147191629, 0.828150842


---------
Индекс I0: 53; I0: -1.48034
Индекс U0: 8; U0: 0.30595
Начальные условия: [15.165447355085933, 0.5873699024243355, 0.4272877413816828]
Спектр: [0.0026647896283262555, -0.1235900510917997, -4.657337045304176]
Последняя точка: [4.079829798695269, 0.8286575328525597, 0.4260042701377623]

---------
Индекс I0: 54; I0: -1.481885
Индекс U0: 8; U0: 0.30595
Начальные условия: [4.079829798695269, 0.8286575328525597, 0.4260042701377623]
Спектр: [0.00517075686419516, -0.09247756424605942, -4.67262460577579]
Последняя точка: [7.1210313313724525, 0.8158819727284619, 0.43290209490316517]

---------
Индекс I0: 55; I0: -1.48343
Индекс U0: 8; U0: 0.30595
Начальные условия: [7.1210313313724525, 0.8158819727284619, 0.43290209490316517]
Спектр: [0.005362961705162067, -0.06087940339804576, -4.688386220364108]
Последняя точка: [4.157922605150482, 0.7880541273435622, 0.4194754381341099]

---------
Индекс I0: 56; I0: -1.484975
Индекс U0: 8; U0: 0.30595
Начальные условия: [4.157922605150482, 0.7880541


---------
Индекс I0: 81; I0: -1.5236
Индекс U0: 8; U0: 0.30595
Начальные условия: [4.9965361060825035, 0.7502384928199577, 0.41573579003219285]
Спектр: [0.0057274712070700595, -2.271934865758686, -2.2804232145862326]
Последняя точка: [8.398505928500725, 0.6701783403961146, 0.417691033244893]

---------
Индекс I0: 82; I0: -1.525145
Индекс U0: 8; U0: 0.30595
Начальные условия: [8.398505928500725, 0.6701783403961146, 0.417691033244893]
Спектр: [0.006162792753631058, -2.2675904312855684, -2.2758884972371787]
Последняя точка: [6.095381485761101, 0.7930711763903721, 0.4279502966101535]

---------
Индекс I0: 83; I0: -1.52669
Индекс U0: 8; U0: 0.30595
Начальные условия: [6.095381485761101, 0.7930711763903721, 0.4279502966101535]
Спектр: [0.0022989721886553533, -2.2621738482750646, -2.271415800332747]
Последняя точка: [4.651241641310696, 0.8580487621023157, 0.4372760761914001]

---------
Индекс I0: 84; I0: -1.528235
Индекс U0: 8; U0: 0.30595
Начальные условия: [4.651241641310696, 0.85804876210

Последняя точка: [3.188089304078259, 0.863162946025728, 0.43249506330293636]

---------
Индекс I0: 109; I0: -1.56686
Индекс U0: 8; U0: 0.30595
Начальные условия: [3.188089304078259, 0.863162946025728, 0.43249506330293636]
Спектр: [0.006339030554395624, -0.1368175064223914, -4.1159127423912985]
Последняя точка: [7.7658502760410695, 0.6785755967902806, 0.418340626843978]

---------
Индекс I0: 110; I0: -1.568405
Индекс U0: 8; U0: 0.30595
Начальные условия: [7.7658502760410695, 0.6785755967902806, 0.418340626843978]
Спектр: [0.003261476631504051, -0.00694367736476362, -4.228444564590556]
Последняя точка: [8.05698665512463, 0.649464055610567, 0.4308475665442261]

---------
Индекс I0: 111; I0: -1.56995
Индекс U0: 8; U0: 0.30595
Начальные условия: [8.05698665512463, 0.649464055610567, 0.4308475665442261]
Спектр: [0.004456937165581949, -0.6799675293913969, -3.5440910440908526]
Последняя точка: [9.66634825943378, 0.6657318967049424, 0.41878004492360893]

---------
Индекс I0: 112; I0: -1.571495


Excessive output truncated after 524350 bytes.